In [1]:
import numpy as np
import pandas as pd
import zoneinfo
import tzdata
import sys
sys.path.append("../")
from src.data.make_dataset import STORM_EVENTS_COMBINED_PATH

In [2]:
combined_files = list(STORM_EVENTS_COMBINED_PATH.glob("*.csv"))
for c_f in combined_files:
    print(c_f.name)

StormEvents_details-all.csv
StormEvents_fatalities-all.csv
StormEvents_locations-all.csv


In [36]:
sample_event_details_df = pd.read_csv(
    STORM_EVENTS_COMBINED_PATH / "StormEvents_details-all.csv",
    dtype={
        "BEGIN_YEARMONTH": str,
        "BEGIN_DAY": str,
        "BEGIN_TIME": str,
        "END_YEARMONTH": str,
        "END_DAY": str,
        "END_TIME": str,
        "BEGIN_DATE_TIME": str,
        "END_DATE_TIME": str
    },
    usecols=lambda col: col not in ["CATEGORY"],
    low_memory=False,
    nrows=5000
)
print(sample_event_details_df.shape)
sample_event_details_df.head()

(5000, 50)


,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,195004,28,1445,195004,28,1445,NaN,10096222,OKLAHOMA,40,...,0,NaN,NaN,35.12,-99.20,35.17,-99.20,NaN,NaN,PUB
1,195004,29,1530,195004,29,1530,NaN,10120412,TEXAS,48,...,0,NaN,NaN,31.90,-98.60,31.73,-98.60,NaN,NaN,PUB
2,195007,5,1800,195007,5,1800,NaN,10104927,PENNSYLVANIA,42,...,0,NaN,NaN,40.58,-75.70,40.65,-75.47,NaN,NaN,PUB
3,195007,5,1830,195007,5,1830,NaN,10104928,PENNSYLVANIA,42,...,0,NaN,NaN,40.60,-76.75,NaN,NaN,NaN,NaN,PUB
4,195007,24,1440,195007,24,1440,NaN,10104929,PENNSYLVANIA,42,...,0,NaN,NaN,41.63,-79.68,NaN,NaN,NaN,NaN,PUB


In [37]:
sample_event_details_df.CZ_TIMEZONE.unique()

array(['CST', 'MST', 'EST', 'PST', 'UNK', 'CDT'], dtype=object)

In [38]:
time_zone_map = {
    "CST": zoneinfo.ZoneInfo("America/Chicago"),
    "MST": zoneinfo.ZoneInfo("America/Denver"),
    "EST": zoneinfo.ZoneInfo("America/New_York"),
    "PST": zoneinfo.ZoneInfo("America/Los_Angeles"),
    "UNK": zoneinfo.ZoneInfo("America/New_York"),
    "CDT": zoneinfo.ZoneInfo("America/Chicago")
}

In [39]:
def scale_str_costs(cost_string) -> int:
    if cost_string[-1] == "K":
        return float(cost_string[:-1]) * 1000
    elif cost_string[-1] == "M":
        return float(cost_string[:-1]) * 1000000
    
    
example_cost_strings = [
    "10.00K",
    "0.00K",
    "10.00M",
    "5.24K",
    "24.8M"
]

example_answers = [
    10000,
    0,
    10000000,
    5240,
    24800000
]

assert all([
    scale_str_costs(cost_str) == answer for cost_str, answer in zip(
        example_cost_strings, example_answers
    )
])

In [40]:
def format_event_details(raw_df: pd.DataFrame) -> pd.DataFrame:
    global time_zone_map
    # Format the input raw columns
    formatted_begin_yrmo = pd.to_datetime(raw_df.BEGIN_YEARMONTH, format="%Y%m")
    formatted_end_yrmo =   pd.to_datetime(raw_df.END_YEARMONTH, format="%Y%m")
    formatted_begin_time = pd.to_datetime(raw_df.BEGIN_TIME.apply(lambda x: x.zfill(2)), format="%H%M")
    formatted_end_time =   pd.to_datetime(raw_df.END_TIME.apply(lambda x: x.zfill(4)), format="%H%M")
    formatted_begin_datetime = pd.to_datetime(raw_df.BEGIN_DATE_TIME, format="%d-%b-%y %H:%M:%S")
    formatted_end_datetime = pd.to_datetime(raw_df.END_DATE_TIME, format="%d-%b-%y %H:%M:%S")
    formatted_time_zone = raw_df.CZ_TIMEZONE.apply(time_zone_map.get)
    formatted_damage_property = raw_df.DAMAGE_PROPERTY.astype(str).apply(scale_str_costs).astype("Int64")
    formatted_damage_crops = raw_df.DAMAGE_CROPS.astype(str).apply(scale_str_costs).astype("Int64")
    # Bring them all together
    formatted_df = pd.concat(
        [
            formatted_begin_yrmo, 
            formatted_end_yrmo, 
            formatted_begin_time,
            formatted_end_time,
            formatted_begin_datetime,
            formatted_end_datetime,
            formatted_time_zone,
            formatted_damage_property,
            formatted_damage_crops
        ],
        axis=1
    )
    return formatted_df
    
    
sample_formatted_raw_columns = format_event_details(sample_event_details_df)
sample_formatted_raw_columns.head()

,BEGIN_YEARMONTH,END_YEARMONTH,BEGIN_TIME,END_TIME,BEGIN_DATE_TIME,END_DATE_TIME,CZ_TIMEZONE,DAMAGE_PROPERTY,DAMAGE_CROPS
0,1950-04-01,1950-04-01,1900-01-01 14:45:00,1900-01-01 14:45:00,2050-04-28 14:45:00,2050-04-28 14:45:00,America/Chicago,250000,<NA>
1,1950-04-01,1950-04-01,1900-01-01 15:30:00,1900-01-01 15:30:00,2050-04-29 15:30:00,2050-04-29 15:30:00,America/Chicago,25000,<NA>
2,1950-07-01,1950-07-01,1900-01-01 18:00:00,1900-01-01 18:00:00,2050-07-05 18:00:00,2050-07-05 18:00:00,America/Chicago,25000,<NA>
3,1950-07-01,1950-07-01,1900-01-01 18:30:00,1900-01-01 18:30:00,2050-07-05 18:30:00,2050-07-05 18:30:00,America/Chicago,2500,<NA>
4,1950-07-01,1950-07-01,1900-01-01 14:40:00,1900-01-01 14:40:00,2050-07-24 14:40:00,2050-07-24 14:40:00,America/Chicago,2500,<NA>


In [45]:
sample_formatted_event_details_df = pd.concat(
    [sample_event_details_df.loc[
        :, 
        [
            x for x in sample_event_details_df.columns if x not in sample_formatted_raw_columns.columns
        ]    
    ],
     sample_formatted_raw_columns],
    axis=1
)
sample_formatted_event_details_df.columns.intersection(sample_event_details_df.columns).size == sample_event_details_df.columns.size

True